In [ ]:
# conda install -y gensim
# run cell
# conda uninstall -y boto
# conda install -y boto

import glob
import os
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
from gensim.models.wrappers import FastText
import pandas as pd
import pymorphy2
import re
from tqdm import tqdm
import _pickle as cPickle

## Настройки

In [ ]:
# Путь к папке с csv-файлами корпуса
corpora_path = './data/2017.12.09-1959-Polarity-Job1943-ru-RU/Reuters_russian_articles/'

# Путь к модели эмбеддингов
# model_path = './models/news.model.bin'  # маленький корпус
# model_path = '../models/ruwikiruscorpora.model.bin'  # большой корпус
model_path = 'rutenten11_8.bin'  # большой корпус со стоп-словами

results_path = 'rutent3'  # директория, куда сохранять результаты

# Количество файлов
FILE_NUMBER = 18000

EMBEDDING_SIZE = 100  # размер эмбеддинга
MAX_SEQ_LENGTH = 30  # максимальное количество слов в предложении
N_BATCH = 128  # размер батча

TRAIN_SPLIT = 0.7  # доля обучающей выборки

## Загрузка эмбеддингов

In [ ]:
# Берем все csv-файлы из нужной директории
directory = glob.glob(os.path.join(corpora_path, '*.csv'))
assert(len(directory) > 0)
len(directory)

In [ ]:
# embeddings_vectors = KeyedVectors.load_word2vec_format(model_path, binary=True)
embeddings_vectors = FastText.load_fasttext_format(model_path)

In [ ]:
# # Преобразуем объект EuclidianKeyedVectors в dict (для второго корпуса)

# embeddings_dict = {}
# for key in embeddings_vectors.vocab:
#     raw_word = key.split("_")[0]
#     embeddings_dict[raw_word] = embeddings_vectors[key]

In [ ]:
embeddings_dict = embeddings_vectors

In [ ]:
# Размер эмбеддинга
EMBEDDING_SIZE = len(embeddings_dict['не'])

# Вектора, обозначающие является ли слово cue или нет
CUE_VECTOR = np.ones(EMBEDDING_SIZE)
NOT_CUE_VECTOR = np.zeros(EMBEDDING_SIZE)

# Пустой вектор
ZERO_VECTOR = np.zeros(EMBEDDING_SIZE)

## Обработка данных

In [ ]:
# Количество предложений (только для теста)
sentenses_count = 0

# Слова, которых нет в словаре эмбеддингов (только для теста)
unsuccessful_embeddings = []

In [ ]:
def prepare_word(word):
    
    word = word.lower()

    # Если слово содержит цифры или другие посторонние символы, выкинем его
    pattern = re.compile("[^а-яА-Яё]")
    if pattern.match(word) != None:
        return False
    
    return word

In [ ]:
'''
Для предложения data создает матрицу эмбеддингов и меток, где номер строки отвечает номеру слова.
'''

def generate_batch(data):

    # Матрица эмбеддингов, номер строки -- это номер слова
    embeddings = []

    # Cue-матрица, номер строки -- это номер слова
    cues = []
    
    # Вектор ответов для каждого слова
    targets = []
    
    # Текст предложения
    text = ""
    
    # Сколько слов получилось по факту (после выкидывания неудавшихся эмбеддингов)
    n_words = 0

    for row_number in range(len(data)):

        # Для каждого слова получаем эмбеддинг и записываем в матрицу
        if (not pd.isnull(data['Text'][row_number])):

            word_orig = data['Text'][row_number].split("=")[1]
            word = prepare_word(word_orig)
            if (not word):
                continue

            # Пытаемся получить эмбеддинг слова
            try:
                # Записываем в матрицу эмбеддингов
                embeddings.append(embeddings_dict[word])
                #embeddings = np.append(embeddings, [embeddings_dict[word]], axis=0)
                
                if (word == 'не' or word == 'нет'):
                    cue_vector = CUE_VECTOR
                else:
                    cue_vector = NOT_CUE_VECTOR
                    
                # Записываем в матрицу cue
                cues.append(cue_vector)

                if (pd.isnull(data['Tags'][row_number])):
                    target = [0, 1]
                else:
                    target = [1, 0]

                # Записываем в вектор ответов
                targets.append(target)
                
                text += word_orig + " | "
                n_words += 1

            except KeyError:
                # В случае неудачи сохраняем в список неудавшихся слов
                unsuccessful_embeddings.append(word)
                
    # Padding
    for padding_index in range(MAX_SEQ_LENGTH - n_words):
        embeddings.append(ZERO_VECTOR)
        cues.append(ZERO_VECTOR)
        targets.append([0, 0])
    
    # Ограничиваем длину предложения величиной MAX_SEQ_LENGTH
    embeddings = np.array(embeddings[:MAX_SEQ_LENGTH])
    cues = np.array(cues[:MAX_SEQ_LENGTH])
    targets = np.array(targets[:MAX_SEQ_LENGTH])
    
    return np.concatenate((embeddings, cues), axis=1), targets, text

In [ ]:
'''
Генерация массива данных размерности (количество предложений, максимальная длина предложения, 2 * размер эмбеддинга)
Вектора эмбеддингов и меток конкатенируются в один вектор длины 2 * EMBEDDING_SIZE

targets: (количество предложений, максимальная длина предложения, 2)
[0, 1] - слово не принадлежит классу отрицания
[1, 0] - слово принадлежит классу отрицания
'''

# Матрица сконкатенированных embeddings и cues
embedding_batch = []

# Матрица ответов
target_batch = []

# Список текстов предложений
texts = []

# Проходимся по всем файлам
for filename in tqdm(directory[:FILE_NUMBER]):

    # Считаем из файла в Pandas Dataframe
    file = pd.read_csv(filename, sep='\t', names=['Offset', 'Text', 'Tags'], skip_blank_lines=False)
    
    sentence = pd.DataFrame()
    words_in_sentence_count = 0
    
    for row_number in range(len(file)):

        # Предложения разделены пустой строкой, поэтому чтобы посчитать количество 
        # предложений, посчитаем количество пустых строк и потом прибавим 1
        if (words_in_sentence_count > MAX_SEQ_LENGTH or pd.isnull(file['Offset'][row_number])):
            
            words_in_sentence_count = 0
            is_new_sentense = True
            
            # Добавляем получившиеся эмбеддинги предложения в общий список
            sent_embeddings, sent_targets, sent_text = generate_batch(sentence)
            
            assert(sent_embeddings.shape == (MAX_SEQ_LENGTH, 2 * EMBEDDING_SIZE))
            assert(sent_targets.shape == (MAX_SEQ_LENGTH, 2))
            
            embedding_batch.append(sent_embeddings)
            target_batch.append(sent_targets)
            
            texts.append(sent_text)
            
            # Стартуем новое предложение
            sentence = pd.DataFrame()
            
        else:
            sentence = sentence.append(file.iloc[[row_number]], ignore_index=True)
            words_in_sentence_count += 1

In [ ]:
embedding_batch = np.array(embedding_batch)

In [ ]:
target_batch = np.array(target_batch)

In [ ]:
np.array(embedding_batch).shape

In [ ]:
# Слова, для которых не удалось получить эмбеддинги
set(unsuccessful_embeddings)

In [ ]:
n_batch = len(embedding_batch)
n_batch

In [ ]:
X = embedding_batch[:int(n_batch * TRAIN_SPLIT)]
X_test = embedding_batch[int(n_batch * TRAIN_SPLIT):]
y = target_batch[:int(n_batch * TRAIN_SPLIT)]
y_test = target_batch[int(n_batch * TRAIN_SPLIT):]

In [ ]:
print("X shape:", X.shape)
print("y shape:", y.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)
print("texts length:", len(texts))

## Сохранение результатов

In [ ]:
# Сохраним посчитанные матрицы в файлы

cPickle.dump(X, open(os.path.join(results_path, "X_train.pkl"), "wb"))
cPickle.dump(X[:10000], open(os.path.join(results_path, "X_train1.pkl"), "wb"))
cPickle.dump(X[10000:20000], open(os.path.join(results_path, "X_train2.pkl"), "wb"))
cPickle.dump(X[20000:], open(os.path.join(results_path, "X_train3.pkl"), "wb"))
cPickle.dump(y, open(os.path.join(results_path, "y_train.pkl"), "wb"))

In [ ]:
# cPickle.dump(X_test, open("X_test.pkl", "wb"))
# cPickle.dump(y_test, open("y_test.pkl", "wb"))

In [ ]:
cPickle.dump(texts, open(os.path.join(results_path, "texts.pkl"), "wb"))